<a href="https://colab.research.google.com/github/YWchris/Effect-of-bias-and-cluster-analysis/blob/main/Random_sample_cataract_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

### Random sample the same number of images as each of the cluster from the training set



In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd 
df = pd.read_csv('gdrive/My Drive/cartact.csv')
len(df)

6392

In [ ]:
df1 = df.loc[:, ['Patient Age','Patient Sex', 'Left-Fundus', 'Left-Diagnostic Keywords']]
df1.rename(columns = {'Left-Fundus':'Image', 'Left-Diagnostic Keywords':'Result'}, inplace = True)

dat_left = df1.loc[(df1['Result'] == 'cataract')|(df1['Result'] == 'normal fundus')]

df2 = df.loc[:, ['Patient Age','Patient Sex', 'Right-Fundus', 'Right-Diagnostic Keywords']]

df2.rename(columns = {'Right-Fundus':'Image', 'Right-Diagnostic Keywords':'Result'}, inplace = True)
dat_right = df2.loc[(df2['Result'] == 'cataract')|(df2['Result'] == 'normal fundus')]
dat_right.head(10)
data = dat_left.append(dat_right)
len(data)

<ipython-input-4-2838f9dca158>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = dat_left.append(dat_right)


6019

In [ ]:
data_cfe = data[(data["Patient Sex"] == "Female") & (data["Result"] == "cataract")]
data_cm = data[(data["Patient Sex"] == "Male") & (data["Result"] == "cataract")]
# data_cfe = data_cfe.iloc[:220]
print(len(data_cfe), len(data_cm))
data_nfe = data[(data["Patient Sex"] == "Female") & (data["Result"] == "normal fundus")]
data_nfe = data_nfe.iloc[:298]
data_nm = data[(data["Patient Sex"] == "Male") & (data["Result"] == "normal fundus")]
data_nm = data_nm.iloc[:220]
print(len(data_nfe), len(data_nm))
frame_data = [data_cfe, data_cm, data_nfe, data_nm]
data = pd.concat(frame_data)
data = data.sample(frac = 1, random_state=42)
len(data)

298 220
298 220


1036

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data, 
                                   test_size = 0.20, 
                                   random_state = 2023,
                                   stratify = data[['Result']])

In [ ]:
# have a look at the proportion of gender in training data
dat1 = train_df[(train_df["Patient Sex"] == "Female") & (train_df["Result"] == "cataract")]
dat2 = train_df[(train_df["Patient Sex"] == "Male") & (train_df["Result"] == "cataract")]

print(len(dat1), len(dat2))

dat3 = train_df[(train_df["Patient Sex"] == "Female") & (train_df["Result"] == "normal fundus")]
dat4 = train_df[(train_df["Patient Sex"] == "Male") & (train_df["Result"] == "normal fundus")]

print(len(dat3), len(dat4))

231 183
241 173


In [ ]:
dat_cataract = train_df[(train_df["Result"] == "cataract")]
dat_normal = train_df[(train_df["Result"] == "normal fundus")]
frame_data = [dat_cataract, dat_normal]
train_df = pd.concat(frame_data)
print(len(train_df))

828


In [ ]:
train_df = train_df.reset_index(drop=True)

In [ ]:
train_df.index.values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [ ]:
# select the images (cataract & normal) for training
train_cataract = train_df.loc[(train_df["Result"] == "cataract")]["Image"].values
train_normal = train_df.loc[(train_df["Result"] == "normal fundus")]["Image"].values

In [ ]:
# select the images (cataract & normal) for testing
test_cataract = test_df.loc[(test_df["Result"] == "cataract")]["Image"].values
test_normal = test_df.loc[(test_df["Result"] == "normal fundus")]["Image"].values

In [ ]:
import torch
import torch.nn.functional as F 
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
REQUIRES_PYTHON = '>=3.5.0'
VERSION = '0.1.0'
from torch import optim, nn
from torchvision import models, transforms

In [ ]:
data_path='/content/drive/MyDrive/dataimage'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On which device we are on:{}".format(device))

from google.colab import drive
drive.mount('/content/drive')

On which device we are on:cpu
Mounted at /content/drive


In [ ]:
# create training dataset, with x=images, y =label(1 is cataract, 0 is normal)
from pandas.core.common import random_state
import cv2
import glob
from PIL import Image
image_size=224
labels = []
dataset = []
random.seed(2023)
def create_dataset(image_category,label):
    for img in tqdm(image_category):
        image_path = os.path.join(data_path,img)
        try:
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)
            image = cv2.resize(image,(image_size,image_size))
        except:
            continue
        
        dataset.append([image,label])
    return dataset

dataset = create_dataset(train_cataract,1)

dataset = create_dataset(train_normal,0)
x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3) #features
y = np.array([i[1] for i in dataset]) #labels
len(x)
len(y)

  0%|          | 0/414 [00:00<?, ?it/s]

  0%|          | 0/414 [00:00<?, ?it/s]

828

In [ ]:
x.shape[0]

828

In [ ]:
# create testing dataset, with x=images, y =label(1 is cataract, 0 is normal)
data_path='/content/drive/MyDrive/dataimage'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On which device we are on:{}".format(device))

from google.colab import drive
drive.mount('/content/drive')

from pandas.core.common import random_state
import cv2
import glob
image_size=224
labels = []
dataset = []
random.seed(1)
def create_dataset(image_category,label):
    for img in tqdm(image_category):
        image_path = os.path.join(data_path,img)
        try:
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)
            image = cv2.resize(image,(image_size,image_size))
        except:
            continue
        
        dataset.append([np.array(image),np.array(label)])
    
    return dataset

dataset = create_dataset(test_cataract,1)

dataset = create_dataset(test_normal,0)

x_testall = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3) #features
y_testall = np.array([i[1] for i in dataset]) #labels
len(x_testall)
len(y_testall)

On which device we are on:cpu
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

208

In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.9.0

Found existing installation: tensorflow 2.9.0
Uninstalling tensorflow-2.9.0:
  Successfully uninstalled tensorflow-2.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-2.9.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511.7 MB)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

datagen.fit(x)

In [ ]:
# Augmentation: Training data
batch_size = 8
num_augmented = 828

augmented_images = np.zeros((num_augmented, image_size, image_size, 3))
for i in range(num_augmented):
    img = x[i % len(x)]  # select an image from the original dataset
    augmented_images[i] = datagen.random_transform(img)

x_aug = np.concatenate((x, augmented_images), axis=0)
y_aug = np.concatenate((y, y[:num_augmented]), axis=0)
y_aug[:len(y)] = y  # repeat the original output data to match the number of samples in x_aug

In [ ]:
# Import base model efficientNetB0 for feature extraction
import numpy as np
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import EfficientNetB0
from keras.optimizers import Adam
from keras.utils import np_utils


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 441
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False 
        
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    checkpoint = ModelCheckpoint("sample3_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    
    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


Epoch 1/10
56/56 [==============================] - ETA: 0s - loss: 0.6803 - accuracy: 0.8912
Epoch 1: val_accuracy improved from -inf to 0.93750, saving model to sample3_efficientnet.h5
56/56 [==============================] - 54s 836ms/step - loss: 0.6803 - accuracy: 0.8912 - val_loss: 0.7463 - val_accuracy: 0.9375
Epoch 2/10
56/56 [==============================] - ETA: 0s - loss: 0.1516 - accuracy: 0.9683
Epoch 2: val_accuracy improved from 0.93750 to 0.94231, saving model to sample3_efficientnet.h5
56/56 [==============================] - 48s 859ms/step - loss: 0.1516 - accuracy: 0.9683 - val_loss: 0.5796 - val_accuracy: 0.9423
Epoch 3/10
56/56 [==============================] - ETA: 0s - loss: 0.1431 - accuracy: 0.9660
Epoch 3: val_accuracy improved from 0.94231 to 0.95673, saving model to sample3_efficientnet.h5
56/56 [==============================] - 43s 769ms/step - loss: 0.1431 - accuracy: 0.9660 - val_loss: 0.6412 - val_accuracy: 0.9567
Epoch 4/10
56/56 [===================

In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 671
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False 
        
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    checkpoint = ModelCheckpoint("sample3_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    
    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


Epoch 1/10
84/84 [==============================] - ETA: 0s - loss: 0.4913 - accuracy: 0.9106
Epoch 1: val_accuracy improved from -inf to 0.90385, saving model to sample3_efficientnet.h5
84/84 [==============================] - 71s 774ms/step - loss: 0.4913 - accuracy: 0.9106 - val_loss: 0.4714 - val_accuracy: 0.9038
Epoch 2/10
84/84 [==============================] - ETA: 0s - loss: 0.3079 - accuracy: 0.9434
Epoch 2: val_accuracy improved from 0.90385 to 0.90865, saving model to sample3_efficientnet.h5
84/84 [==============================] - 63s 755ms/step - loss: 0.3079 - accuracy: 0.9434 - val_loss: 0.7881 - val_accuracy: 0.9087
Epoch 3/10
84/84 [==============================] - ETA: 0s - loss: 0.1649 - accuracy: 0.9702
Epoch 3: val_accuracy did not improve from 0.90865
84/84 [==============================] - 59s 702ms/step - loss: 0.1649 - accuracy: 0.9702 - val_loss: 0.7498 - val_accuracy: 0.9087
Epoch 4/10
84/84 [==============================] - ETA: 0s - loss: 0.0577 - accur

In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 274
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False 
        
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    checkpoint = ModelCheckpoint("sample3_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    
    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


Epoch 1/10
35/35 [==============================] - ETA: 0s - loss: 0.5647 - accuracy: 0.8759
Epoch 1: val_accuracy improved from -inf to 0.92788, saving model to sample3_efficientnet.h5
35/35 [==============================] - 43s 1s/step - loss: 0.5647 - accuracy: 0.8759 - val_loss: 0.7873 - val_accuracy: 0.9279
Epoch 2/10
35/35 [==============================] - ETA: 0s - loss: 0.6469 - accuracy: 0.9343
Epoch 2: val_accuracy improved from 0.92788 to 0.94712, saving model to sample3_efficientnet.h5
35/35 [==============================] - 34s 987ms/step - loss: 0.6469 - accuracy: 0.9343 - val_loss: 0.6924 - val_accuracy: 0.9471
Epoch 3/10
35/35 [==============================] - ETA: 0s - loss: 0.1446 - accuracy: 0.9745
Epoch 3: val_accuracy did not improve from 0.94712
35/35 [==============================] - 34s 997ms/step - loss: 0.1446 - accuracy: 0.9745 - val_loss: 0.5050 - val_accuracy: 0.9375
Epoch 4/10
35/35 [==============================] - ETA: 0s - loss: 0.0615 - accuracy

In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 42
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x_aug.shape[0]), size=cluster_size, replace=False)
    x_sample = x_aug[sample_indices]
    y_sample = y_aug[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False 
        
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    checkpoint = ModelCheckpoint("sample3_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    
    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


Epoch 1/10
6/6 [==============================] - ETA: 0s - loss: 0.6916 - accuracy: 0.7143
Epoch 1: val_accuracy improved from -inf to 0.93269, saving model to sample3_efficientnet.h5
6/6 [==============================] - 26s 4s/step - loss: 0.6916 - accuracy: 0.7143 - val_loss: 0.3200 - val_accuracy: 0.9327
Epoch 2/10
6/6 [==============================] - ETA: 0s - loss: 0.9232 - accuracy: 0.8571
Epoch 2: val_accuracy did not improve from 0.93269
6/6 [==============================] - 17s 3s/step - loss: 0.9232 - accuracy: 0.8571 - val_loss: 0.4302 - val_accuracy: 0.9087
Epoch 3/10
6/6 [==============================] - ETA: 0s - loss: 0.2083 - accuracy: 0.9524
Epoch 3: val_accuracy did not improve from 0.93269
6/6 [==============================] - 18s 3s/step - loss: 0.2083 - accuracy: 0.9524 - val_loss: 0.6465 - val_accuracy: 0.9231
Epoch 4/10
6/6 [==============================] - ETA: 0s - loss: 0.2555 - accuracy: 0.9048
Epoch 4: val_accuracy did not improve from 0.93269
Resto

In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 149
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x.shape[0]), size=cluster_size, replace=False)
    x_sample = x[sample_indices]
    y_sample = y[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False 
        
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    checkpoint = ModelCheckpoint("sample3_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    
    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 34
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x.shape[0]), size=cluster_size, replace=False)
    x_sample = x[sample_indices]
    y_sample = y[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False 
        
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    checkpoint = ModelCheckpoint("sample_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    
    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 135
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x.shape[0]), size=cluster_size, replace=False)
    x_sample = x[sample_indices]
    y_sample = y[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False 
        
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    checkpoint = ModelCheckpoint("samplec_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    
    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 13
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x.shape[0]), size=cluster_size, replace=False)
    x_sample = x[sample_indices]
    y_sample = y[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False 
        
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    checkpoint = ModelCheckpoint("samplec_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    
    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, confusion_matrix

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 117
results = []
auc_scores = []
sensitivities = []
specificities = []

for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x.shape[0]), size=cluster_size, replace=False)
    x_sample = x[sample_indices]
    y_sample = y[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
        layer.trainable = False 
        
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    checkpoint = ModelCheckpoint("samplec_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    # Train the model
    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    
    # Test the model and calculate the metrics
    y_pred = model.predict(x_testall)
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    auc_score = roc_auc_score(y_testall, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_testall, y_pred.round()).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Store the metrics in lists
    results.append(test_acc)
    auc_scores.append(auc_score)
    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Calculate the average metrics
avg_test_acc = np.mean(results)
avg_auc = np.mean(auc_scores)
avg_sensitivity = np.mean(sensitivities)
avg_specificity = np.mean(specificities)

print('Average testing accuracy:', avg_test_acc)
print('Average AUC:', avg_auc)
print('Average sensitivity:', avg_sensitivity)
print('Average specificity:', avg_specificity)


In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 21
results = []
for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x.shape[0]), size=cluster_size, replace=False)
    x_sample = x[sample_indices]
    y_sample = y[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
      layer.trainable = False 
      
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    checkpoint = ModelCheckpoint("sample3_efficientnet.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='auto', restore_best_weights=True)

    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    results.append(test_acc)

    # Calculate the average testing accuracy
avg_test_acc = np.mean(results)
print('Average testing accuracy:', avg_test_acc)


0.9452

Average testing accuracy: 0.9288461565971374

In [ ]:
import numpy as np
import keras
from keras.applications.efficientnet import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Randomly sample the dataset with the same size as the cluster size
cluster_size = 179
results = []
for i in range(5):
    # Randomly select a sample from the dataset
    sample_indices = np.random.choice(range(x.shape[0]), size=cluster_size, replace=False)
    x_sample = x[sample_indices]
    y_sample = y[sample_indices]
    
    sample_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    # Freeze all layers of the pre-trained model
    for layer in sample_model1.layers:
      layer.trainable = False 
      
    model = Sequential()
    model.add(sample_model1)
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    checkpoint = ModelCheckpoint("sample4_efficientnet.h5", monitor="val_loss", verbose=1, save_best_only=True, mode='min')
    earlystop = EarlyStopping(monitor="val_loss", patience=5, verbose=1, mode='auto', restore_best_weights=True)

    model.fit(x_sample, y_sample, batch_size=8, epochs=10, validation_data=(x_testall, y_testall), verbose=1, callbacks=[checkpoint, earlystop])
    test_loss, test_acc = model.evaluate(x_testall, y_testall)
    results.append(test_acc)

    # Calculate the average testing accuracy
avg_test_acc = np.mean(results)
print('Average testing accuracy:', avg_test_acc)


0.9375

Average testing accuracy: 0.9480769276618958